우리는 crfsuite 라는 C++ 기반 crf package를 파이썬 환경에서 이용할 수 있도록 래핑을 한, pycrfsuite를 이용하여 띄어쓰기 교정기를 만듭니다. 

이 과정은 (1) 주어진 문장을 feature로 만드는 과정과, (2) 이를 모델 학습에 이용하는 과정, (3) 그리고 학습된 모델을 이용하여 띄어쓰기 교정을 하는 과정으로 이뤄져 있습니다. 

(1) feature를 만드는 과정에 따라 Named Entity Recognition과 같은 태깅 작업도 수행할 수 있으며, [pycrfsuite 튜토리얼][pycrfsuite_tutorial]에는 영어의 NER 예제가 실려 있습니다. 

사용할 예제는 영화 '라라랜드'의 리뷰 15,599개 입니다. 

[pycrfsuite_tutorial]: https://github.com/scrapinghub/python-crfsuite/blob/master/examples/CoNLL%202002.ipynb

In [1]:
import pycrfsuite
import sys

from config import dataset_dir
sys.path.append('{}/lovit_textmining_dataset/'.format(dataset_dir))

Dataset version
[navermovie_comments.data] is latest (0.0.1)
[navermovie_comments.models] is latest (0.0.1)
[navernews_10days.data] is latest (0.0.1)
[navernews_10days.models] is latest (0.0.1)


In [3]:
from navermovie_comments import load_movie_comments

idxs, docs, rates = load_movie_comments(idxs='134963')
len(docs)

15599

이 문장을 예시로하여 설명을 하겠습니다. 

In [4]:
docs[0]

'시사회에서 보고왔습니다동화와 재즈뮤지컬의 만남 지루하지않고 재밌습니다'

먼저 이용할 feature의 templates을 만들어야 합니다. 아래에 출력된 list of tuple을 만들 건데, 이후의 재활용성을 고려하여 함수로 만들어 두었습니다. 

    [(-2, 0), (-2, 1), (-2, 2), (-1, 1), (-1, 2), (0, 2)]

함수에서 begin, end index의 부호가 같을 경우에는 템플릿으로 만들지 않는 부분이 있습니다. 이 부분은 띄어쓰기 알고리즘의 특징 때문에 넣어둔 부분입니다. 이 부분으로 인하여 글자의 앞부분만 본다던지 (-2,-1), 뒷부분만 보는 경우 (1,2)가 사라집니다. 만약 NER 작업을 하신다면 이 때에는 앞 부분의 글자만으로도 도움이 되기 때문에 이 부분을 삭제하시는 걸 추천합니다. 띄어쓰기 교정의 경우에 이 한 쪽의 글자들만 보는 것이 왜 위험한지는 soyspacing package를 설명하며 이야기하겠습니다. 

    if b * e > 0:
        continue
        

In [5]:
def generate_templates(begin=-2, end=2, min_range_length=3, max_range_length=5):
    templates = []
    for b in range(begin, end):
        for e in range(b, end+1):
            length = (e - b + 1)
            if length < min_range_length or length > max_range_length: 
                continue
            if b * e > 0:
                continue
            templates.append((b, e))
    return templates

templates = generate_templates()
print(templates)

[(-2, 0), (-2, 1), (-2, 2), (-1, 1), (-1, 2), (0, 2)]


앞서 만든 templates을 이용하여 character sequence로부터 feature를 만드는 함수를 만듭니다. 이후에도 여러번 쓸 클래스여서 \__call\__을 구현해뒀습니다. 

'테스트문장입니다'라는 글자열에 대하여 아래와 같은 features가 만들어집니다. 

    [[('X[0,2]', '예문입')],
     [('X[-1,1]', '예문입'), ('X[-1,2]', '예문입니'), ('X[0,2]', '문입니')],
     [('X[-2,0]', '예문입'),
      ('X[-2,1]', '예문입니')
     ...

"('X[0,2]', '테스트')"는 지금 글자는 "테"이고, 뒤의 두 글자가 "스트"라는 의미입니다. 

In [6]:
class CharacterFeatureTransformer:
    def __init__(self, templates):
        self.templates = templates
    def __call__(self, chars, tags=None):
        x =[]
        for i in range(len(chars)):
            xi = []
            e_max = len(chars)
            for t in self.templates:
                b = i + t[0]
                e = i + t[1] + 1
                if b < 0 or e > e_max:
                    continue
                xi.append(('X[%d,%d]' % (t[0], t[1]), chars[b:e]))
            x.append(xi)
        return x
    
transformer = CharacterFeatureTransformer(templates)
transformer('예문입니다')

[[('X[0,2]', '예문입')],
 [('X[-1,1]', '예문입'), ('X[-1,2]', '예문입니'), ('X[0,2]', '문입니')],
 [('X[-2,0]', '예문입'),
  ('X[-2,1]', '예문입니'),
  ('X[-2,2]', '예문입니다'),
  ('X[-1,1]', '문입니'),
  ('X[-1,2]', '문입니다'),
  ('X[0,2]', '입니다')],
 [('X[-2,0]', '문입니'), ('X[-2,1]', '문입니다'), ('X[-1,1]', '입니다')],
 [('X[-2,0]', '입니다')]]

문장을 (1) 글자열과 (2) 띄어쓰기 태그로 분리하는 함수를 만듭니다. character sequence와 label sequence를 만들기 위함입니다. 1은 띄어쓴다, 0은 붙여쓴다를 의미하며, 문장의 맨 끝은 반드시 띄어쓰는 태그를 부여합니다. 

In [7]:
def sent_to_chartags(sent, nonspace=0, space=1):
    chars = sent.replace(' ','')
    if not chars:
        return '', []

    tags = [nonspace]*(len(chars) - 1) + [space]
    idx = 0
    for c in sent:
        if c == ' ':
            tags[idx-1] = space
        else:
            idx += 1
    return chars, tags

sent_to_chartags('예문 입니다')

('예문입니다', [0, 1, 0, 0, 1])

sent_to_xy는 한 문장이 들어왔을 때, (1) feature로 바꿔주는 transformer를 이용하여 문장의 글자열 chars로부터 feature를 만들고, (2) 각 글자에 해당하는 띄어쓰기 label인 y를 만들어 return 합니다. 

sent_to_xy를 이용하면 아래와 같이 학습가능한 형태의 x와 y가 만들어 짐을 볼 수 있습니다. 

In [8]:
def sent_to_xy(sent, feature_transformer):
    chars, tags = sent_to_chartags(sent)
    x = [['%s=%s' % (xij[0], xij[1]) for xij in xi] for xi in feature_transformer(chars, tags)]
    y = [str(t) for t in tags]
    return x, y

x,y = sent_to_xy('예문 입니다', transformer)

In [9]:
x

[['X[0,2]=예문입'],
 ['X[-1,1]=예문입', 'X[-1,2]=예문입니', 'X[0,2]=문입니'],
 ['X[-2,0]=예문입',
  'X[-2,1]=예문입니',
  'X[-2,2]=예문입니다',
  'X[-1,1]=문입니',
  'X[-1,2]=문입니다',
  'X[0,2]=입니다'],
 ['X[-2,0]=문입니', 'X[-2,1]=문입니다', 'X[-1,1]=입니다'],
 ['X[-2,0]=입니다']]

학습은 Trainer를 만든 뒤, 한문장씩 feature x와 label y를 append 해줍니다. 이 때에는 반드시 x와 y의 길이가 같아야 합니다. 우리가 만든 예제에는 빈 글자열이 있어서 이를 확인하여 걸러내기 위한 len(x) != len(y) 부분을 넣어뒀습니다. 

    trainer = pycrfsuite.Trainer(verbose=False)
    ...
    trainer.append(x, y)

pycrfsuite를 이용할 때 자주 이용하는 parameters 입니다. 기본으로 설정된 iteration 횟수가 많이 큽니다. 이를 적당한 수준으로 조절해주며, c1, c2를 이용하여 L1, L2 regularization을 걸 수 있습니다. c1 == 0 이면, L2 regularization 만 이용한다는 의미입니다. 

가장 주의해야 할 parameter 중 하나로, feature.minfreq를 잘 설정해야 합니다. 기본값은 0으로 되어있기 때문에 한 번이라도 등장한 모든 feature를 이용하게 됩니다. 이 경우에는 (1) overfitting 이 일어날 수 있고, (2) 데이터가 조금이라도 커지면 수천만차원의 벡터 공간을 만들게 됩니다 (3천만차원 logstic regression을 학습한다고 상상해 보십시요). feature.minfreq를 설정하여 학습이 가능한 상태가 되도록 만들어 줍니다. 

pycrfsuite는 crfsuite package를 파이썬으로 쓸 수 있도록 만든 래핑 패키지 이므로, parameter 메뉴얼은 [링크](crfsuite)를 참조해 주세요

pycrfsuite는 crfsuite의 C++ 코드를 그대로 실행합니다. 이 코드는 반드시 학습된 모델을 bin 파일로 저장하도록 되어있는데, 이 때문에 trainer.train(model_fname)으로 학습된 모델을 일단 저장해야 합니다. 

[crfsuite]: http://www.chokkan.org/software/crfsuite/manual.html

In [10]:
params = {
    'max_iterations':50, # default 1000
    'c1': 0, # L1 regularization, default 1
    'c2': 1, # L2 regularization, default 2
    'feature.minfreq': 3 # minimum frequency of feature. default 0 (without pruning)
}

model_fname = 'crfsuite_spacing.model'

trainer = pycrfsuite.Trainer(verbose=False)
for sent in docs:
    x, y = sent_to_xy(sent, transformer)
    trainer.append(x, y)
trainer.set_params(params)
trainer.train(model_fname)

저장된 모델은 pycrfsuite.Tagger()로 tagger를 만든 뒤, tagger.open()으로 읽어올 수 있습니다. 

In [11]:
tagger = pycrfsuite.Tagger()
tagger.open(model_fname)

교정할 문장은 sent와 transformer를 입력받은 뒤, 글자열에 대하여 feature x를 만듭니다. 이를 이용하여 tagging을 수행하고, y_pred의 값에 따라 띄어쓰기를 교정합니다. 

In [12]:
def correct(sent, feature_transformer):
    char = sent.replace(' ','')
    x, y = sent_to_xy(sent, feature_transformer)
    y_pred = tagger.tag(x)
    return ''.join([c if tag == '0' else c + ' ' for c, tag in zip(char, y_pred)])

correct('이건정말로좋은영화였다', transformer)

'이건 정말로 좋은영화였다 '

(feature, class)에 대한 coefficients는 아래처럼 확인할 수 있습니다. 

weights는 {(attribute, class):weight} 형식의 dict 입니다. 

In [13]:
debugger = tagger.info()
weights = debugger.state_features

('X[-2,0]=시사회', '1')는 '시사회'라는 글자 뒤에 띄어쓴다라고 판단하는 점수가 0.094868이라는 의미입니다. 

In [14]:
list(weights.items())[:10]

[(('X[0,2]=시사회', '0'), 0.143225),
 (('X[-1,1]=시사회', '0'), 0.352454),
 (('X[-1,2]=시사회에', '0'), 0.085102),
 (('X[0,2]=사회에', '0'), 0.105701),
 (('X[-2,0]=시사회', '0'), -0.094868),
 (('X[-2,0]=시사회', '1'), 0.094868),
 (('X[-2,1]=시사회에', '0'), 0.086027),
 (('X[-2,2]=시사회에서', '0'), 0.086027),
 (('X[-1,1]=사회에', '0'), 0.106155),
 (('X[-1,2]=사회에서', '0'), 0.106155)]

### pycrfsuite를 이용한 띄어쓰기 교정기 래핑 코드

위 부분들에 대하여 세부 함수들을 간소화 시키고, feature.minfreq 부분의 계산 시 메모리를 많이 이용하여 큰 부하를 주는 부분을 덜 신경쓰기 위해 github.com/lovit/pycrfsuite_spacing 에 미리 위 함수들을 패키지화 시켜뒀습니다. 

사용법은 README와 [tutorial][pycrfsuite_spacing_tutorial]에 올려뒀습니다. 

설치법은 git clone을 하셔도 되며, pip install pycrfsuite-spacing 을 하셔도 됩니다. dependency에 의하여 python-crfsuite >= 0.9.2 도 설치됩니다.

[pycrfsuite_spacing_tutorial]: https://github.com/lovit/pycrfsuite_spacing/blob/master/tutorials/tutorial.ipynb

In [15]:
from pycrfsuite_spacing import TemplateGenerator
from pycrfsuite_spacing import CharacterFeatureTransformer
from pprint import pprint

In [16]:
import pycrfsuite_spacing
pycrfsuite_spacing.__version__

'1.0.2'

In [17]:
templates = TemplateGenerator(
    begin=-2, 
    end=2,
    min_range_length=3,
    max_range_length=3)

to_feature = CharacterFeatureTransformer(templates)

In [18]:
list(to_feature.templates)

[(-2, 0), (-1, 1), (0, 2)]

In [19]:
x, y = sent_to_xy('이것도 너프해 보시지', to_feature)
pprint(x)
print(y)

[['X[0,2]=이것도'],
 ['X[-1,1]=이것도', 'X[0,2]=것도너'],
 ['X[-2,0]=이것도', 'X[-1,1]=것도너', 'X[0,2]=도너프'],
 ['X[-2,0]=것도너', 'X[-1,1]=도너프', 'X[0,2]=너프해'],
 ['X[-2,0]=도너프', 'X[-1,1]=너프해', 'X[0,2]=프해보'],
 ['X[-2,0]=너프해', 'X[-1,1]=프해보', 'X[0,2]=해보시'],
 ['X[-2,0]=프해보', 'X[-1,1]=해보시', 'X[0,2]=보시지'],
 ['X[-2,0]=해보시', 'X[-1,1]=보시지'],
 ['X[-2,0]=보시지']]
['0', '0', '1', '0', '0', '1', '0', '0', '1']


In [20]:
from pycrfsuite_spacing import PyCRFSuiteSpacing

correct = PyCRFSuiteSpacing(
    to_feature,
    feature_minfreq=5, 
    max_iterations=100,
    l1_cost=1.0,
    l2_cost=1.0
)
correct.train(docs, 'package_test.crfsuite')

In [21]:
correct('이건진짜좋은영화라라랜드진짜좋은영화')

'이건 진짜 좋은 영화 라라랜드 진짜 좋은 영화'

In [22]:
correct('라라랜드데이터로봉준호감독의플란다스의개를이야기하면이해하기어렵습니다')

'라라랜드 데이터로봉준호감독의 플란다스의개를이야기 하면 이해하기 어렵습니다'